<h1 style='text-align: center; font-size: 48px; background: #DEDEDE; padding: 25px 10px; border-radius: 25px;'>Basketball Webscraper</h1>

<br>


    
Simple webscraper for NBA data using three different sources.


<h1 style= 'text-align: center; font-size: 32px; padding: 25px 10px; background: #DEDEDE; border-radius: 25px;'>Table of Contents</h1>



## 1. [Setup](#setup)
### &nbsp; &nbsp; &nbsp; 1.1  [Import Packages](#packages)
### &nbsp; &nbsp; &nbsp; 1.2  [PostgreSQL Connection](#sql)
## 2. [Webscraping](#webscraping)
### &nbsp; &nbsp; &nbsp; 2.1  [Player Data](#player)
### &nbsp; &nbsp; &nbsp; 2.2  [Team Data](#team)
### &nbsp; &nbsp; &nbsp; 2.3  [MVP Data](#mvp)
## 3. [Upload Datasets](#upload)


<a class='anchor' id= setup></a>
<h2 style= 'text-align: center; font-size: 32px; padding: 25px 10px; background: #DEDEDE; border-radius: 25px;'>Setup</h2>


<a class='anchor' id= packages></a>
<h3>Import Packages</h3> 

In [130]:
import pandas as pd
import numpy as np
import requests
import time
import psycopg2
from sqlalchemy import create_engine
from bs4 import BeautifulSoup
from Keyfile import key_fun #Custom file to hide postgreSQL Server information

<a class='anchor' id= sql></a>
<h3>PostgreSQL Connection</h3> 

In [131]:
key = key_fun()

engine = create_engine('postgresql://' + key['username'] + ':' + key['pwd'] + '@localhost:' + 
                       key['port_id'] + '/' + key['database'])

<a class='anchor' id= 'webscraping'></a>
<h2 style= 'text-align: center; font-size: 32px; padding: 25px 10px; background: #DEDEDE; border-radius: 25px;'>Webscraping</h2>

<br>

<center><strong>Sites Scraped</strong></center>

|Data|Site|Link|
|:---|:---|:---:|
| Player | NBA.com |<a href= 'https://www.nba.com/stats/leaders?Season=2022-23&SeasonType=Regular+Season&PerMode=Totals'>Link 1</a>|
| Team | ESPN |<a href = 'https://www.espn.com/nba/standings'>Link 2</a></li>|
| MVP | Basketball-Reference |<a href = 'https://www.basketball-reference.com/awards/mvp.html'>Link 3</a>|
|<img width=250/>|<img width=250/>|<img width=250/>|


<a class='anchor' id= player></a>
<h3>Player Data</h3>

<br>

<div style= 'padding: 14px 14px;'>
Though it is nice that NBA.com provides an API, we will need to scrape data across multiple pages for different seasons. Thankfully the urls follow a predictable pattern:
    
<br>

<u>stats.nba.com/stats/leagueLeaders?LeagueID=00&PerMode=Totals&Scope=S&Season=**2021-22**&SeasonType=Regular%20Season&StatCategory=PTS</u>    
    
<u>stats.nba.com/stats/leagueLeaders?LeagueID=00&PerMode=Totals&Scope=S&Season=**2022-23**&SeasonType=Regular%20Season&StatCategory=PTS</u>


<br>

The website contains data from the 1951-52 season through the 2022-23 season as of writing this. Generating a list of the desired dates is simple enough. Now it's possible to iterate over all the desired pages.
</div>

In [132]:
seasons = [str(x) + '-' + str(x+1)[2:] for x in range(1951,2023)]

<div style= 'padding: 14px 14px;'>
    Using the headers from the webpage, create a <strong>header_index</strong> dictionary to keep track of which data value is being collected at each index. The main reason this is used is to simplify iteration later. The same result could be achieved using a list, but dictionary look ups are faster. The <strong>player_index</strong> dictionary organizes the stats being collected to simplify creating a dataframe later.
</div>

In [133]:
page_url = "https://stats.nba.com/stats/leagueLeaders?LeagueID=00&PerMode=Totals&Scope=S&Season=2022-23&SeasonType=Regular%20Season&StatCategory=PTS"
page=requests.get(url = page_url).json()
headers = page['resultSet']['headers']

header_index = {i : header for i, header in enumerate(headers)}
player_data = {header: [] for header in headers}
player_data['SEASON'] = []

<div style= 'padding: 14px 14px;'>
Webpages are scraped one by one with a slight delay between API requests. Stats are uploaded to the player_data dictionary and a <strong>SEASON</strong> column is added to keep track of which season the data represents.
</div>

In [134]:
for season in seasons:
    time.sleep(2)
    
    page_url = "https://stats.nba.com/stats/leagueLeaders?LeagueID=00&PerMode=Totals&Scope=S&Season=" + season + "&SeasonType=Regular%20Season&StatCategory=PTS"
    page=requests.get(url = page_url).json()
    players = page['resultSet']['rowSet']

    for player in players:
        for i, stat in enumerate(player):
            player_data[ header_index[i] ].append(stat)
        player_data[ 'SEASON' ].append(season)

<div style= 'padding: 14px 14px;'>
The dictionary is then converted into a pandas dataframe.
</div>

In [135]:
df_players = pd.DataFrame.from_dict(player_data)
df_players.head()

,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,...,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV,SEASON
0,76056,1,Paul Arizin,1610612744,PHW,66,2939,548,1222,0.448,...,170,NaN,NaN,NaN,250,1674,1786,NaN,NaN,1951-52
1,600012,2,George Mikan,1610612747,MNL,64,2572,545,1414,0.385,...,194,NaN,NaN,NaN,286,1523,1592,NaN,NaN,1951-52
2,600003,3,Bob Cousy,1610612738,BOS,66,2681,512,1388,0.369,...,441,NaN,NaN,NaN,190,1433,1322,NaN,NaN,1951-52
3,77429,4,Ed Macauley,1610612738,BOS,66,2631,384,888,0.432,...,232,NaN,NaN,NaN,174,1264,1396,NaN,NaN,1951-52
4,76514,5,Bob Davies,1610612758,ROC,65,2394,379,990,0.383,...,390,NaN,NaN,NaN,269,1052,935,NaN,NaN,1951-52


<a class='anchor' id= team></a>
<h3>Team Data</h3>

<br>

<div style= 'padding: 14px 14px;'>
The data from the team is also important when selecting MVP. Particularly in recent years, a teams overall record and standing seem to factor into the MVP discussion. The best players on bad teams are often overlooked by their more winning counterparts. That being said, it is necessary to also acquire data from the teams from each season. 
  
<br>
    
The two dictionaries below simplify data collection and organization.
</div>

In [145]:
index2cat= {
    0: 'W',
    1: 'L',
    2: 'PCT',
    3: 'GB',
    4: 'HOME',
    5: 'AWAY',
    6: 'DIV',
    7: 'CONF',
    8: 'PPG',
    9: 'OPPPPG',
    10: 'DIFF',
    11: 'STRK',
    12: 'L10'
}

team_data = {
    'W': [],
    'L': [],
    'PCT': [],
    'GB': [],
    'HOME': [],
    'AWAY': [],
    'DIV': [],
    'CONF': [],
    'PPG': [],
    'OPPPPG': [],
    'DIFF': [],
    'STRK': [],
    'L10': [],
    'TEAM': [],
    'SEASON': []
}

<div style= 'padding: 14px 14px;'>
The webscraping loop for the team data. This loop is similar to one used to collect player data. The main difference is the other loop used an API, whereas the setup of this requires sifting through the html for the desired data.
</div>

In [146]:
seasons = [str(x) for x in range(2003,2024)]
for season in seasons:
    time.sleep(3)
    
    page_url = "https://www.espn.com/nba/standings/_/sort/gamesbehind/dir/asc/season/" + season
    r = requests.get(page_url)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    for team in soup.find_all('abbr'): team_data['TEAM'].append(team.get_text())

    for i, stat in enumerate(soup.find_all('span', class_ = 'stat-cell')):
        team_data[ index2cat[ i%13 ] ].append(stat.get_text())
        
        if i%13 == 0:
            team_data['SEASON'].append( str(int(season)-1) + '-' + season[2:] )
        


<div style= 'padding: 14px 14px;'>
Some of the teams present have abbreviations that are not consistent with those from the player dataset. Since the vision is to later merge these two datasets using SEASON and TEAM as keys, the inconsistencies will need to be corrected.
</div>

In [147]:
remap = {'NJ': 'NJN', 'SA': 'SAS', 'NY':'NYK', 'GS': 'GSW', 'UTAH': 'UTA'}

#2013-14 is first year that New Orleans Hornets became New Orleans Pelicans
#NO -> NOH 2002-03 - 2012-13
#NO -> NOP 2013-14 - *
for i, team in enumerate(team_data['TEAM']):
    if team == 'NO':
        if int(team_data['SEASON'][i][0:4]) < 2013:
            team_data['TEAM'][i] = 'NOH'
        else:
            team_data['TEAM'][i] = 'NOP'

    
df_teams = pd.DataFrame.from_dict(team_data)
df_teams.replace({'TEAM': remap}, inplace=True)
df_teams.head()

,W,L,PCT,GB,HOME,AWAY,DIV,CONF,PPG,OPPPPG,DIFF,STRK,L10,TEAM,SEASON
0,50,32,.610,-,30-11,20-21,19-9,0-0,91.4,87.7,+3.7,L1,4-6,DET,2002-03
1,49,33,.598,1,33-8,16-25,16-8,0-0,95.4,90.1,+5.3,L2,5-5,NJN,2002-03
2,48,34,.585,2,32-9,16-25,19-9,0-0,96.8,93.3,+3.5,W2,6-4,IND,2002-03
3,48,34,.585,2,25-16,23-18,17-7,0-0,96.8,94.5,+2.3,W1,5-5,PHI,2002-03
4,47,35,.573,3,29-12,18-23,17-11,0-0,93.9,91.8,+2.1,W5,7-3,NOH,2002-03


<a class='anchor' id= mvp></a>
<h3>MVP Data</h3>

<div style= 'padding: 14px 14px;'>
The following information is to be added to the mvp dataframe:
    <ul>
        <li>PLAYER</li>
        <li>MVP</li>
        <li>CONSEC_MVP</li>
        <li>TOTAL_MVP</li>
        <li>SEASON</li>       
    </ul>
<br>
    
The MVP data is the easiest to scrape, because all the data is on a single webpage. There's no need to put time between requests.
</div>

In [139]:
mvp_data = {
    'PLAYER': [],
    'MVP': [],
    'SEASON': []
}

In [140]:
url = "https://www.basketball-reference.com/awards/mvp.html"
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
table = soup.find('table')
players = table.find_all('td', attrs={"data-stat": "player"})[::-1]

seasons = [ season for season in range(1955,2023) ]
mvp_data['MVP'] = [ 1 for _ in range( len(players) ) ]

for i, player in enumerate(players):
    player = player.get_text()
    mvp_data['PLAYER'].append(player)
    mvp_data['SEASON'].append(str(seasons[i]) + '-' + str(seasons[i]+1)[2:]) #Key for future merge    

In [141]:
df_mvp = pd.DataFrame.from_dict(mvp_data)
df_mvp.head()

,PLAYER,MVP,SEASON
0,Bob Pettit,1,1955-56
1,Bob Cousy,1,1956-57
2,Bill Russell,1,1957-58
3,Bob Pettit,1,1958-59
4,Wilt Chamberlain,1,1959-60


<a class='anchor' id= 'upload'></a>
<h2 style= 'text-align: center; font-size: 32px; padding: 25px 10px; background: #DEDEDE; border-radius: 25px;'>Upload Datasets to PostgreSQL</h2>

<br>

<div style= 'padding: 14px 14px;'>
Converts dataframes to tables in PostgreSQL database for later use.
</div>

In [142]:
df_players.to_sql('player_data', con = engine, if_exists = 'replace') #player dataset

832

In [148]:
df_teams.to_sql('team_data', con = engine, if_exists = 'replace') #team dataset

628

In [144]:
df_mvp.to_sql('mvp_data', con = engine, if_exists = 'replace')  #mvp dataset

68